In [1]:
import pandas as pd 
import numpy as np 
from utils.text_helpers import *
from layers.attention import AttentionDecoder

/Users/agatawlaszczyk/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Wczytanie i przetworzenie danych
Początkowo wczytujemy dane z przygotowanego pliku. Rozdzielamy je na zbiór treningowy, zbiór walidacyjny i zbiór testowy. 

In [2]:
#do we want to analyze the job_content or job_name? 
dataset_details = {
    'data_focus': 'job_content',
    'max_number_words': 20000,
    'max_seq_len': 250
}

In [3]:
data = Data(dataset_details['data_focus'])

In [4]:
print(len(data.train_X), len(data.val_X), len(data.test_X))
#sprawdzamy czy wszystkie zbiory zawierają te same kategorie 
print(set(pd.unique(data.train_y))==set(pd.unique(data.val_y)) and set(pd.unique(data.val_y))==set(pd.unique(data.test_y)))

17000 3000 1500
True


In [5]:
example_id=24
print("Opis: ", data.train_X.values[example_id])
print("Kategoria: ", data.train_y.values[example_id])

Opis:  firma wenglorz jako jedyna na polskim rynku wykonuje kompleksowo obiekty wytwórni pasz od projektu technologicznego i budowlanego przez produkcję konstrukcji i urządzeń po montaż i uruchomienie. zajmuje się także projektowaniem produkukowaniem oraz montażem kompletnych linii technologicznych wraz z ich rozruchem. . opis stanowiska pracy umiejętność czytania dokumentacji technicznej wykonawczej oraz warsztatowej opracowywanie na podstawie dokumentacji zestawień elementów przeznaczonych do zamówienia opracowanie plików w formacie nc dstv dla maszyn sterowanych numerycznie ulepszanie istniejących rozwiązań technicznych wprowadzenie nowych technologii innowacji usprawnień bieżący nadzór nad wykonywaniem zleceń produkcyjnych kontrola zgodności dokumentacji produkcyjnej z odpowiednimi standardami przepisami oraz normami technicznymi i normami bezpieczeństwa rozwój istniejących rozwiązań technicznych oraz wprowadzanie nowych technologii wymagania wykształcenie wyższe techniczne znajomo

### Przygotowanie tekstu

Zanim stworzymy model i wytrenujemy go, musimy odpowiednio przetworzyć tekst, żeby dostosować reprezentację tekstową do reprezentacji akceptowalnej przez model (tensory). <br>

Kroki preprocessingu: 
1. lemmatyzacja tekstu
2. dodanie tokenu kończącego tekst do każdego wpisu\
3. tokenizacja tekstu
4. text to vector
5. dodanie znaków początkowych i końcowych dla kategorii
6. transformacja kategorii do ich domyślnej reprezentacji dla modelu 


In [6]:
#1 i 2: lemmatyzacja + dodanie tokenu konczacego zdanie
datasets = [data.train_X, data.val_X, data.test_X]
print("Oryginalny tekst: {}".format(data.train_X.values[example_id]))

for i,dataset in enumerate(datasets):
    datasets[i] = dataset.apply(lambda x: data.lemmatize(x) + " <eos>")
print("*"*10)
print("Wynik kroków 1 i 2: {}".format(datasets[0].values[example_id]))    

#3: tokenizacja tekstu 
data.fit_tokenizer(datasets[0].values, max_number_words=dataset_details['max_number_words'])

#4: text to vector
for i, dataset in enumerate(datasets):
    datasets[i] = data.text_to_vector(dataset, max_seq_len=dataset_details['max_seq_len'])
print("*"*10)
print("Wynik kroków 3 i 4: {}".format(datasets[0][example_id]))

Oryginalny tekst: firma wenglorz jako jedyna na polskim rynku wykonuje kompleksowo obiekty wytwórni pasz od projektu technologicznego i budowlanego przez produkcję konstrukcji i urządzeń po montaż i uruchomienie. zajmuje się także projektowaniem produkukowaniem oraz montażem kompletnych linii technologicznych wraz z ich rozruchem. . opis stanowiska pracy umiejętność czytania dokumentacji technicznej wykonawczej oraz warsztatowej opracowywanie na podstawie dokumentacji zestawień elementów przeznaczonych do zamówienia opracowanie plików w formacie nc dstv dla maszyn sterowanych numerycznie ulepszanie istniejących rozwiązań technicznych wprowadzenie nowych technologii innowacji usprawnień bieżący nadzór nad wykonywaniem zleceń produkcyjnych kontrola zgodności dokumentacji produkcyjnej z odpowiednimi standardami przepisami oraz normami technicznymi i normami bezpieczeństwa rozwój istniejących rozwiązań technicznych oraz wprowadzanie nowych technologii wymagania wykształcenie wyższe technic

In [7]:
#5: dodanie znaków początkowych i końcowych do kategorii 
labels = [data.train_y, data.val_y, data.test_y]
print("Initial labels: {}".format(labels[0].values[example_id]))
for i,label_set in enumerate(labels): 
    labels[i] = label_set.apply(lambda x: ("\t " + x.replace(" ", "_") + " \n").split(" "))
print("After step 5: {}".format(labels[0].values[example_id]))
print("*"*10)

#6 transformacja do reprezentacji domyślnej dla modelu 
labels_input = []
labels_output = []
for i, label_set in enumerate(labels): 
    transformed_input, transformed_output = data.transform_labels(label_set)
    labels_input.append(transformed_input)
    labels_output.append(transformed_output)
print("Input after step 6: {}".format(labels_input[0][example_id]))
print("Output after step 6: {}".format(labels_output[0][example_id]))
    

Initial labels: Produkcja
After step 5: ['\t', 'Produkcja', '\n']
**********
Input after step 6: [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Output after step 6: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


Mając przystosowane reprezentacje tekstowe, możemy stworzyć schemat modelu. Model zaimplementowany jest w Kerasie. Składa się z trzech głównych komponentów: 
- encodera
- decodera
- atencji. 


In [8]:
from keras.models import load_model, Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Input, Concatenate, TimeDistributed, GRU
from keras.initializers import Constant

In [9]:
#additional network info
network = Network({
    'embedding_dim': 300,
    'embeddings': load_word_embeddings('./data/wiki.pl.vec'),
    'train_embeddings': True,
    'latent_dim': dataset_details['max_seq_len'],
    'dropout': 0.5, 
    'num_decoder_tokens': 37,
    'batch_size': 500, 
    'num_epochs': 5,
    'return_probabilities': False,
    'max_labels': 37
})


In [10]:
embed_matrix = generate_embedding_matrix(network.embeddings, data.tokenizer.word_index)
print(embed_matrix.shape)

(20001, 300)


In [14]:
encoder_inputs = Input(shape=(None,), name='encoder_input')
embedding_layer = Embedding(input_dim=dataset_details['max_number_words']+1,
                            output_dim=network.embedding_dim,
                            embeddings_initializer = Constant(embed_matrix),
                            trainable=network.train_embeddings, name='embeddings')  
embedded_sequences = embedding_layer(encoder_inputs)
encoder = Bidirectional(LSTM(network.latent_dim, return_sequences=True, dropout=network.dropout), name='encoder')
states =  encoder(embedded_sequences)
outputs_true = Input(shape=(None, None,), dtype='int64', name='decoder_input')
decoder_outputs = AttentionDecoder(network.latent_dim*2, network.max_labels, return_probabilities=network.return_probabilities, name='attention')([states, outputs_true], use_teacher_forcing=False)
model = Model([encoder_inputs, outputs_true], decoder_outputs)

    
    

In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

In [16]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
embeddings (Embedding)          (None, None, 300)    6000300     encoder_input[0][0]              
__________________________________________________________________________________________________
encoder (Bidirectional)         (None, None, 500)    1102000     embeddings[0][0]                 
__________________________________________________________________________________________________
decoder_input (InputLayer)      (None, None, None)   0                                            
__________________________________________________________________________________________________
attention 

In [26]:
import keras.callbacks as callback 
history = model.fit([datasets[0], labels_input[0]], labels_output[0],
              batch_size=network.batch_size,
              epochs=network.num_epochs,
              validation_data=[[datasets[1], labels_input[1]], labels_output[1]], verbose=1, 
                    callbacks=[callback.EarlyStopping(patience=2, restore_best_weights = True)])

Train on 17000 samples, validate on 3000 samples
Epoch 1/5
17000/17000 [==============================] - 516s 30ms/step - loss: 0.1041 - categorical_accuracy: 0.9682 - val_loss: 0.2515 - val_categorical_accuracy: 0.9352
Epoch 2/5
17000/17000 [==============================] - 492s 29ms/step - loss: 0.0959 - categorical_accuracy: 0.9704 - val_loss: 0.2638 - val_categorical_accuracy: 0.9310
Epoch 3/5
17000/17000 [==============================] - 492s 29ms/step - loss: 0.0832 - categorical_accuracy: 0.9740 - val_loss: 0.2742 - val_categorical_accuracy: 0.9322


In [24]:
save = True
path_to_model = './data/model_seq2seq_{}.h5'.format(dataset_details['data_focus'])
if save:
    model.save(path_to_model)
    model.save_weights(path_to_model.replace('.h5', '_weights.h5'))

In [19]:
# 0.8898 (bez atencji po 5 epokach na val)
# 0.9244 (z atencją po 5 epokach na val)
# 0.9332 (z atencją po 10 epokach na val)
# 0.9368 (z atencją po 15 epokach)

In [25]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
embeddings (Embedding)          (None, None, 300)    6000300     encoder_input[0][0]              
__________________________________________________________________________________________________
encoder (Bidirectional)         (None, None, 500)    1102000     embeddings[0][0]                 
__________________________________________________________________________________________________
decoder_input (InputLayer)      (None, None, None)   0                                            
__________________________________________________________________________________________________
attention 